## 1. Basic Model Creation and Automatic Data Conversion
### Pydantic’s BaseModel makes it simple to define data models that automatically convert and validate types.

In [20]:
# %pip install -r requirements.txt

In [2]:
from pydantic import BaseModel
from datetime import datetime
from typing import List, Optional

class User(BaseModel):
    id: int
    name: str
    signup_ts: Optional[datetime] = None
    friends: List[int] = []

# Create a user instance; note that the datetime string is automatically parsed.
user = User(id=123, name="John Doe", signup_ts="2020-07-25 12:22", friends=[1, 2, 3])
print(user)

id=123 name='John Doe' signup_ts=datetime.datetime(2020, 7, 25, 12, 22) friends=[1, 2, 3]


### Key Points:

### Type conversion: Pydantic converts compatible types automatically (e.g., string to datetime).
### Default values: Optional fields and default values work as expected.



## 2. Handling Validation Errors
### When the data doesn’t match the specified types, Pydantic raises a ValidationError. You can catch and inspect these errors.

In [3]:
from pydantic import ValidationError

try:
    # Intentionally passing a wrong type for id
    User(id="not_an_int", name="Jane")
except ValidationError as e:
    print(e.json())
# Output the error in a readable format

[{"type":"int_parsing","loc":["id"],"msg":"Input should be a valid integer, unable to parse string as an integer","input":"not_an_int","url":"https://errors.pydantic.dev/2.11/v/int_parsing"}]


### Key Points:

### Error reporting: Pydantic provides detailed JSON error messages showing which field failed validation.

## 3. Nested Models
### Models can be nested inside one another. Pydantic handles conversion and validation recursively.

In [4]:
class Address(BaseModel):
    street: str
    city: str

class UserWithAddress(BaseModel):
    name: str
    address: Address

# Providing a dict for the nested Address; it gets converted automatically.
user = UserWithAddress(name="Alice", address={"street": "Main", "city": "Wonderland"})
print(user)
# Accessing nested attributes

name='Alice' address=Address(street='Main', city='Wonderland')


### Key Points:

### Deep validation: Nested dictionaries are converted to their respective Pydantic model types.

## 4. Custom Field Validation Using Validators
### You can add custom validation logic to any field using the @validator decorator.

In [5]:
from pydantic import validator

class UserWithEmail(BaseModel):
    username: str
    email: str

    @validator('email')
    def email_must_contain_at_symbol(cls, v):
        if '@' not in v:
            raise ValueError('must contain an @ symbol')
        return v

# Valid case:
user = UserWithEmail(username="alice", email="alice@example.com")
print(user)

# Invalid case will raise an error:
try:
    UserWithEmail(username="bob", email="bobexample.com")
except ValidationError as e:
    print(e)
# Output the error in a readable format
# Custom error message

username='alice' email='alice@example.com'
1 validation error for UserWithEmail
email
  Value error, must contain an @ symbol [type=value_error, input_value='bobexample.com', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error


/var/folders/t2/55vgl7rd2pd84zg2rhmyz6ww0000gn/T/ipykernel_3844/4281625855.py:7: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator('email')


### Key Points:

### Field-specific logic: Use @validator to enforce additional rules beyond type checking.

## 5. Root Validators for Inter-field Validation
### When validation logic depends on multiple fields, use a root_validator.

In [13]:
from pydantic import BaseModel, ValidationError, model_validator

class UserWithPassword(BaseModel):
    password: str
    confirm_password: str

    @model_validator(mode="after")
    def check_passwords_match(cls, model: "UserWithPassword") -> "UserWithPassword":
        if model.password != model.confirm_password:
            raise ValueError("passwords do not match")
        return model

# Correct usage:
user = UserWithPassword(password="secret123", confirm_password="secret123")
print(user)

# Incorrect usage: output the error in a readable format
try:
    UserWithPassword(password="secret123", confirm_password="different")
except ValidationError as e:
    print(e)


password='secret123' confirm_password='secret123'
1 validation error for UserWithPassword
  Value error, passwords do not match [type=value_error, input_value={'password': 'secret123',..._password': 'different'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error


### Key Points:

### Cross-field validation: root_validator checks multiple fields together, making sure the combined state is valid.

## 6. Parsing Data from Different Sources
### Pydantic provides utility methods to parse data from various formats.

In [8]:
# Parsing from a Python dict:
user_obj = UserWithEmail.parse_obj({"username": "alice", "email": "alice@example.com"})
print(user_obj)

# Converting a model to JSON:
user_json = user_obj.json()
print(user_json)


username='alice' email='alice@example.com'
{"username":"alice","email":"alice@example.com"}


/var/folders/t2/55vgl7rd2pd84zg2rhmyz6ww0000gn/T/ipykernel_3844/1387851750.py:2: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  user_obj = UserWithEmail.parse_obj({"username": "alice", "email": "alice@example.com"})
/var/folders/t2/55vgl7rd2pd84zg2rhmyz6ww0000gn/T/ipykernel_3844/1387851750.py:6: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  user_json = user_obj.json()


### Key Points:

### Flexibility: Easily convert between dicts, JSON strings, and Pydantic models.

## 7. Settings Management with BaseSettings
### For managing configurations, Pydantic’s BaseSettings allows you to load values from environment variables and .env files.

In [24]:
from pydantic_settings import BaseSettings

class Settings(BaseSettings):
    app_name: str
    admin_email: str

    class Config:
        env_file = ".env"

# Create settings instance; values are taken from environment or the .env file.
settings = Settings()
print(settings.dict())

{'app_name': 'Demonstration', 'admin_email': 'admin@example.com'}


/var/folders/t2/55vgl7rd2pd84zg2rhmyz6ww0000gn/T/ipykernel_3844/2920283302.py:12: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(settings.dict())


## Key Points:

### Configuration management: Load and validate environment configurations seamlessly with type safety.